In [1]:
from src.FunctionsProperties import *

In [ ]:
# Separate files to surface
N__ = [5000, 10000, 20000, 40000]
dim = [1,2,3,4]

for j in N__:
    for i in dim:
        parms =  [k for k in list_all_folders(j,i)]        
        for l in range(len(parms)):
            all_properties_dataframe(j, i, parms[l][0], parms[l][1])

In [ ]:
alpha_a = [0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0]
alpha_g = [0.1, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0]

for n in N__:
    for d in dim:
        for g in alpha_g:
            for a in alpha_a:
                alpha_G = list_all_folders_for_alpha_fixed(n, d , a, g, alpha_g_variable=True)
                create_all_properties_file(n, d, a, alpha_G, alpha_g_variable=True)

    for d in dim:
        for a in alpha_a:
            for g in alpha_g:
                alpha_A = list_all_folders_for_alpha_fixed(n, d , a, g, alpha_g_variable=False)
                create_all_properties_file(n, d, alpha_A, g, alpha_g_variable=False)

In [2]:
all_data = pd.read_csv(f"../../data/all_data_filter.csv", sep = ',')
all_data.head()

,alpha_a,alpha_g,n_size,dim,short_mean,diameter_mean,ass_coeff_mean,short_err,diameter_err,ass_coeff_err,n_samples
0,3.0,0.1,20000,4,95.185838,253.619048,-0.269746,3.787748,9.508011,0.001209,21
1,2.0,0.1,20000,4,62.093195,163.761905,-0.242202,1.259514,4.412593,0.001956,21
2,9.0,4.0,20000,4,14.208324,34.714286,-0.018333,0.092836,0.526185,0.001656,21
3,2.0,8.0,20000,4,9.689103,27.095238,-0.044656,0.105900,0.266411,0.001598,21
4,7.0,4.0,20000,4,13.373938,32.904762,-0.040303,0.116930,0.377364,0.001891,21


In [8]:
all_data['n_size'].unique()

array([20000, 40000, 10000,  5000])

In [9]:
# Dict to load beta_values with parameters
properties_dict = {"alpha_a":[], "alpha_g":[], "dim":[],
                    "beta_short":[], "beta_diameter":[],"beta_assortativity":[],
                   "beta_short_err":[], "beta_diameter_err":[],"beta_assortativity_err":[]}

# Create tuple with alpha_a, alpha_g, dim from dataframe
index_dict_data = [(i, j, k) for i,j,k in zip(all_data['alpha_a'],
                                              all_data['alpha_g'],
                                              all_data['dim'])]
# Extract unique tuples using set
unique_tuples = set(index_dict_data)

# List of N_values
List_N = [5000, 10000, 20000, 40000]


# run for unique values of (alpha_a, alpha_g, dim)
for element in unique_tuples:
    
    # Append parameter values in dictionary
    properties_dict["alpha_a"].append(element[0])
    properties_dict["alpha_g"].append(element[1])
    properties_dict["dim"].append(element[2])
    
    # lists to help store Betas and errors
    short_mean, short_mean_err = [], []
    diameter_mean, diameter_mean_err = [], []
    ass_coeff_mean, ass_coeff_mean_err = [], []
    
    # Run the code to each n in List_N
    for n in List_N:
        # Select the index with unique (alpha_a, alpha_g, dim) for each n
        idx_values_N = all_data[(all_data['alpha_a'] == element[0]) & 
                           (all_data['alpha_g'] == element[1]) &
                           (all_data['dim'] == element[2]) & 
                           (all_data['n_size'] == n)].index[0]
        
        # Append properties in lists
        short_mean.append(all_data["short_mean"].iloc[idx_values_N])
        short_mean_err.append(all_data["short_err"].iloc[idx_values_N])

        diameter_mean.append(all_data["diameter_mean"].iloc[idx_values_N])
        diameter_mean_err.append(all_data["diameter_err"].iloc[idx_values_N])

        ass_coeff_mean.append(all_data["ass_coeff_mean"].iloc[idx_values_N])
        ass_coeff_mean_err.append(all_data["ass_coeff_err"].iloc[idx_values_N])
    
    # Calcule of linear regression with errors (beta, beta_err)
    beta_short, beta_short_err = linear_regression(np.log10(List_N), 
                                                   np.array(short_mean), np.array(short_mean_err), Parameter = True)
    
    beta_diameter, beta_diameter_err = linear_regression(np.log10(List_N), 
                                                np.array(diameter_mean), np.array(diameter_mean_err), Parameter = True)
    
    beta_ass, beta_ass_err = linear_regression(np.log10(List_N), 
                                                np.array(ass_coeff_mean), np.array(ass_coeff_mean_err), Parameter = True)
    
    # Append values in dictionary
    properties_dict["beta_short"].append(beta_short)
    properties_dict["beta_short_err"].append(beta_short_err)

    properties_dict["beta_diameter"].append(beta_diameter)
    properties_dict["beta_diameter_err"].append(beta_diameter_err)

    properties_dict["beta_assortativity"].append(beta_ass)
    properties_dict["beta_assortativity_err"].append(beta_ass_err)

# Create and save DataFrame with dictionary data
df = pd.DataFrame(data=properties_dict)
df.to_csv(f"../../data/all_beta.csv", delimiter = ',', mode = "w+", index = False)


IndexError: index 0 is out of bounds for axis 0 with size 0